In [ ]:
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv

pasta_atual = os.getcwd()
print(f"O Python está rodando a partir de: {pasta_atual}")

if pasta_atual.endswith('notebooks'):
    prefixo = '../'
else:
    prefixo = ''

caminho_env = f'{prefixo}.env'
caminho_csv = f'{prefixo}data/raw/WA_Fn-UseC_-Telco-Customer-Churn.csv'

load_dotenv(dotenv_path=caminho_env)

DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_NAME')

print(f"Tentando ler o CSV no caminho exato: {caminho_csv}")

df_clientes = pd.read_csv(caminho_csv)
print("✅ 1. Dados originais lidos com sucesso!")


df_clientes.columns = [
    'customer_id', 'gender', 'senior_citizen', 'partner', 'dependents',
    'tenure', 'phone_service', 'multiple_lines', 'internet_service',
    'online_security', 'online_backup', 'device_protection', 'tech_support',
    'streaming_tv', 'streaming_movies', 'contract', 'paperless_billing',
    'payment_method', 'monthly_charges', 'total_charges', 'churn'
]

df_clientes['total_charges'] = pd.to_numeric(df_clientes['total_charges'], errors='coerce')
df_clientes['total_charges'] = df_clientes['total_charges'].fillna(0)
print("✅ 2. Dados limpos e padronizados!")

db_url = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
engine = create_engine(db_url)

print("⏳ 3. Enviando dados para o banco...")
df_clientes.to_sql('raw_customers', engine, if_exists='append', index=False)

print("🚀 SUCESSO TOTAL! Banco de dados populado com os dados limpos!")

O Python está rodando a partir de: c:\Users\USER\OneDrive\Área de Trabalho\P\Projetos\analise-churn-clientes\notebooks
Tentando ler o CSV no caminho exato: ../data/raw/WA_Fn-UseC_-Telco-Customer-Churn.csv
✅ 1. Dados originais lidos com sucesso!
✅ 2. Dados limpos e padronizados!
⏳ 3. Enviando dados para o banco...
🚀 SUCESSO TOTAL! Banco de dados populado com os dados limpos!
